In [112]:
import pandas as pd
df=pd.read_csv('04_02_2019.csv')

In [113]:
rename_stages = {
    'THORAIPAKK|THURAIPAKK': 'THORAIPAKKAM',
    'ANNA UNIVE': 'ANNA UNIV',
    'MOOTAIKARA': 'M K CHAVADI',
    'SHOZHANGAN|SHOZHINGAN': 'SHOLINGANALLUR',
    'KELAMBAKKA':'KELAMBAKKAM',
    'KOMAN NGR': 'KUMARAN NG',
    'M.K. CHAVA': 'M K CHAVADI',
    'WOMENS POL': 'WPTC',
    'THIRUPPORU|TIRUPPORUR': 'THIRUPORUR',
}
df = df.replace(rename_stages, regex=True)

df['Source'] = df['Source'].str.strip()
df['From Stage'] = df['From Stage'].str.strip()
df['To Stage'] = df['To Stage'].str.strip()
df.to_csv('04-02-19.csv')

In [114]:
df=pd.read_csv('04-02-19.csv')
df=df[df['Schedule Name'].str.contains(r'\b519\b')]
cols=['Schedule Name','Adult','From Stage','To Stage','Trip Start Time','Trip End Time','Source','Destination']
df=df[cols]

In [115]:
df=df[(df['Source']=='T.NAGAR') & (df['Destination']=='THIRUPORUR')]

# Convert the trip start time and end time columns to datetime objects
df['Trip Start Time'] = pd.to_datetime(df['Trip Start Time'], format='%H:%M:%S')
df['Trip End Time'] = pd.to_datetime(df['Trip End Time'], format='%H:%M:%S')

# Filter the rows where trip starts between the given start time and end time
start_time = pd.to_datetime('08:00:00').time()
end_time = pd.to_datetime('09:00:00').time()
df = df[(df['Trip Start Time'].dt.time >= start_time) & 
        (df['Trip Start Time'].dt.time < end_time)]

df

,Schedule Name,Adult,From Stage,To Stage,Trip Start Time,Trip End Time,Source,Destination
48903,519-A1-AS-MDE,1,T.NAGAR,ANNA UNIV,1900-01-01 08:30:00,1900-01-01 10:46:00,T.NAGAR,THIRUPORUR
48904,519-A1-AS-MDE,1,T.NAGAR,KANDANCHAV,1900-01-01 08:30:00,1900-01-01 10:46:00,T.NAGAR,THIRUPORUR
48905,519-A1-AS-MDE,1,T.NAGAR,SRP TOOLS,1900-01-01 08:30:00,1900-01-01 10:46:00,T.NAGAR,THIRUPORUR
48906,519-A1-AS-MDE,1,T.NAGAR,NAVALUR,1900-01-01 08:30:00,1900-01-01 10:46:00,T.NAGAR,THIRUPORUR
48907,519-A1-AS-MDE,1,T.NAGAR,M K CHAVADI,1900-01-01 08:30:00,1900-01-01 10:46:00,T.NAGAR,THIRUPORUR
...,...,...,...,...,...,...,...,...
53166,519-Z1-AS-MDE,1,KANDANCHAV,SIPCOT,1900-01-01 08:05:00,1900-01-01 10:23:00,T.NAGAR,THIRUPORUR
53171,519-Z1-AS-MDE,1,KANDANCHAV,SIPCOT,1900-01-01 08:05:00,1900-01-01 10:23:00,T.NAGAR,THIRUPORUR
53172,519-Z1-AS-MDE,1,THORAIPAKKAM,NAVALUR,1900-01-01 08:05:00,1900-01-01 10:23:00,T.NAGAR,THIRUPORUR
53173,519-Z1-AS-MDE,1,THORAIPAKKAM,NAVALUR,1900-01-01 08:05:00,1900-01-01 10:23:00,T.NAGAR,THIRUPORUR


In [116]:
bus_stages = [
    'T.NAGAR', 'SAIDAPET', 'ANNA UNIV', 'WPTC', 'SRP TOOLS',
    'KANDANCHAV', 'THORAIPAKKAM', 'M K CHAVADI', 'KARAPAKKAM',
    'SHOLINGANALLUR', 'KUMARAN NG', 'CHEMMANCHE', 'NAVALUR',
    'SIPCOT', 'CHURCH', 'PAL. CHEMI', 'HINDUSTAN', 'KELAMBAKKAM',
    'KOMAN NAGAR', 'ENGG', 'CHENGAMMAL', 'KALAVAKKAM','THIRUPORUR'
]

stage_mapping = {stage: i for i, stage in enumerate(bus_stages)}

df['From Stage'] = df['From Stage'].map(stage_mapping)
df['To Stage'] = df['To Stage'].map(stage_mapping)

df.dropna(subset=['From Stage', 'To Stage'], inplace=True)

#create an empty OD matrix
od_matrix = pd.DataFrame(index=range(len(bus_stages)), columns=range(len(bus_stages))).fillna(0)

# populating the matrix 
for index, row in df.iterrows():
    source = int(row['From Stage'])
    destination = int(row['To Stage'])
    adult_count = row['Adult']
    od_matrix.loc[source, destination] += adult_count if not pd.isna(adult_count) else 0

od_matrix['Boarding'] = od_matrix.sum(axis=1)
od_matrix.loc['ALIGHTING'] = od_matrix.sum(axis=0)


#display the entire df
print(od_matrix.to_string())

           0  1  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  Boarding
0          0  3  1  10  28  19   9  21  12  16   3   2   2   0   7   0   1   4   0   0   0   1   3       142
1          0  0  1   7  10  14   9   3   4   4   0   1   1   0   2   1   0   1   0   0   0   0   0        58
2          0  0  0   1   9   4   2   3   3   0   0   0   1   0   3   0   0   4   0   0   0   0   0        30
3          0  0  0   0   0   4   3   5  12   3   5   1   6   0   3   0   1   2   0   0   0   0   4        49
4          0  0  0   0   0   0   1   0   0   2   0   0   1   0   0   0   0   1   0   0   0   0   1         6
5          0  0  0   0   0   0   0   0   3   1   2   2   2   2   0   0   0   1   0   0   0   0   2        15
6          0  0  0   0   0   0   0   1   0   1   1   1  12   0   3   1   1   2   0   0   0   0   2        25
7          0  0  0   0   0   0   0   0   0   1   0   1   3   0   0   0   0   5   0   0   0   0   3        13
8          0  0  0 

In [117]:

# Create an empty OD matrix with stage names as indexes and columns
od_matrix = pd.DataFrame(index=bus_stages, columns=bus_stages).fillna(0)

# Populate the matrix 
for index, row in df.iterrows():
    source = bus_stages[int(row['From Stage'])]
    destination = bus_stages[int(row['To Stage'])]
    adult_count = row['Adult']
    od_matrix.loc[source, destination] += adult_count if not pd.isna(adult_count) else 0

# Calculate Boarding and Alighting
od_matrix['Boarding'] = od_matrix.sum(axis=1)
od_matrix.loc['ALIGHTING', :] = od_matrix.sum(axis=0)

# Display the entire OD matrix
od_matrix


,T.NAGAR,SAIDAPET,ANNA UNIV,WPTC,SRP TOOLS,KANDANCHAV,THORAIPAKKAM,M K CHAVADI,KARAPAKKAM,SHOLINGANALLUR,...,CHURCH,PAL. CHEMI,HINDUSTAN,KELAMBAKKAM,KOMAN NAGAR,ENGG,CHENGAMMAL,KALAVAKKAM,THIRUPORUR,Boarding
T.NAGAR,0.0,3.0,1.0,10.0,28.0,19.0,9.0,21.0,12.0,16.0,...,7.0,0.0,1.0,4.0,0.0,0.0,0.0,1.0,3.0,142.0
SAIDAPET,0.0,0.0,1.0,7.0,10.0,14.0,9.0,3.0,4.0,4.0,...,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,58.0
ANNA UNIV,0.0,0.0,0.0,1.0,9.0,4.0,2.0,3.0,3.0,0.0,...,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,30.0
WPTC,0.0,0.0,0.0,0.0,0.0,4.0,3.0,5.0,12.0,3.0,...,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,4.0,49.0
SRP TOOLS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6.0
KANDANCHAV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,15.0
THORAIPAKKAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,3.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,25.0
M K CHAVADI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,13.0
KARAPAKKAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,13.0
SHOLINGANALLUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,6.0,9.0,0.0,0.0,0.0,0.0,6.0,35.0
